# Quantum Neural Network

## Setup

### Libraries Installation

In [2]:
!pip install qiskit
!pip install qiskit_machine_learning
!pip install qiskit-machine-learning[sparse]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.7 MB 4.7 MB/s 
     |████████████████████████████████| 18.0 MB 286 kB/s 
     |████████████████████████████████| 240 kB 25.1 MB/s 
     |████████████████████████████████| 112 kB 59.1 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 1.6 MB 37.6 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 37.5 MB 1.2 MB/s 
     |████████████████████████████████| 943 kB 54.4 MB/s 
     |████████████████████████████████| 4.1 MB 38.0 MB/s 
     |████████████████████████████████| 112 kB 44.2 MB/s 
  Created wheel for qiskit: filename=qiskit-0.37.1-py3-none-any.whl size=12067 sha256=b0d1bf68216797d4fc35e6286d1eeb7a6c453c03d688201172416631ee06522b
  Stored in directory: /root/.cache/pip/wheels/1c/3c/a3/bf8c2

### Libraries Import

In [3]:
import numpy as np
from qiskit.circuit import Parameter
from qiskit import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.opflow import StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient

### Global Constants

In [4]:
# set random seed
algorithm_globals.random_seed = 42

# set method to calculcate expected values
expval = AerPauliExpectation()

# define gradient method
gradient = Gradient()

# define quantum instances (statevector and sample based)
qi_sv = QuantumInstance(Aer.get_backend("aer_simulator_statevector"))

# we set shots to 10 as this will determine the number of samples later on
qi_qasm = QuantumInstance(Aer.get_backend("aer_simulator"), shots = 10)

## Opflow QNN

In [5]:
from qiskit_machine_learning.neural_networks import OpflowQNN

In [6]:
# construct parametrized circuit
params1 = [Parameter("input1"), Parameter("weight1")]
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.ry(params1[0], 0)
qc1.rx(params1[1], 0)
qc_sfn1 = StateFn(qc1)

# construct cost operator
H1 = StateFn(PauliSumOp.from_list([("Z", 1.0), ("X", 1.0)]))

# combine operator and circuit to objective function
op1 = ~H1 @ qc_sfn1

qc1.draw()

┌───┐┌────────────┐┌─────────────┐
q: ┤ H ├┤ Ry(input1) ├┤ Rx(weight1) ├
   └───┘└────────────┘└─────────────┘

In [7]:
# construct OpflowQNN with the operator, the input and weight parameters, the expected value, gradient, and quantum instance
qnn1 = OpflowQNN(op1, [params1[0]], [params1[1]], expval, gradient, qi_sv)

In [8]:
# define random input and weights
input1 = algorithm_globals.random.random(qnn1.num_inputs)
weights1 = algorithm_globals.random.random(qnn1.num_weights)

In [9]:
# QNN forward pass
qnn1.forward(input1, weights1)

array([[0.08242345]])

In [10]:
# QNN batched forward pass
qnn1.forward([input1, input1], weights1)

array([[0.08242345],
       [0.08242345]])

In [11]:
# QNN backward pass
qnn1.backward(input1, weights1)

(None, array([[[0.2970094]]]))

In [12]:
# QNN batched backward pass
qnn1.backward([input1, input1], weights1)

(None, array([[[0.2970094]],
 
        [[0.2970094]]]))

In [13]:
# combining multiple observables in a ListOp also allows to create more complex QNNs
op2 = ListOp([op1, op1])
qnn2 = OpflowQNN(op2, [params1[0]], [params1[1]], expval, gradient, qi_sv)

In [14]:
# QNN forward pass
qnn2.forward(input1, weights1)

array([[0.08242345, 0.08242345]])

In [15]:
# QNN backward pass
qnn2.backward(input1, weights1)

(None, array([[[0.2970094],
         [0.2970094]]]))

## Two Layers QNN

In [16]:
from qiskit_machine_learning.neural_networks import TwoLayerQNN

In [17]:
# specify the number of qubits
num_qubits = 3

In [18]:
# specify the feature map
fm = ZZFeatureMap(num_qubits, reps=2)
fm.draw()

┌───────────────────────────────┐
q_0: ┤0                              ├
     │                               │
q_1: ┤1 ZZFeatureMap(x[0],x[1],x[2]) ├
     │                               │
q_2: ┤2                              ├
     └───────────────────────────────┘

In [19]:
# specify the ansatz
ansatz = RealAmplitudes(num_qubits, reps=1)
ansatz.draw()

┌────────────────────────────────────────────────┐
q_0: ┤0                                               ├
     │                                                │
q_1: ┤1 RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5]) ├
     │                                                │
q_2: ┤2                                               ├
     └────────────────────────────────────────────────┘

In [20]:
# specify the observable
observable = PauliSumOp.from_list([("Z" * num_qubits, 1)])

In [21]:
# define two layer QNN
qnn3 = TwoLayerQNN(
    num_qubits, feature_map=fm, ansatz=ansatz, observable=observable, quantum_instance=qi_sv
)

In [22]:
# define random input and weights
input3 = algorithm_globals.random.random(qnn3.num_inputs)
weights3 = algorithm_globals.random.random(qnn3.num_weights)

In [23]:
# QNN forward pass
qnn3.forward(input3, weights3)

array([[0.18276559]])

In [24]:
# QNN backward pass
qnn3.backward(input3, weights3)

(None, array([[[ 0.10231208,  0.10656571,  0.41017902,  0.16528909,
          -0.27780262,  0.41365763]]]))

## Circuit QNN

In [25]:
from qiskit_machine_learning.neural_networks import CircuitQNN

In [26]:
qc = RealAmplitudes(num_qubits, entanglement="linear", reps=1)
qc.draw()

┌────────────────────────────────────────────────┐
q_0: ┤0                                               ├
     │                                                │
q_1: ┤1 RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5]) ├
     │                                                │
q_2: ┤2                                               ├
     └────────────────────────────────────────────────┘

In [27]:
# specify circuit QNN
qnn4 = CircuitQNN(qc, [], qc.parameters, quantum_instance=qi_qasm)

In [28]:
# define random input and weights
input4 = algorithm_globals.random.random(qnn4.num_inputs)
weights4 = algorithm_globals.random.random(qnn4.num_weights)

In [29]:
# QNN forward pass, returned as a sparse matrix
qnn4.forward(input4, weights4)

array([[0.3, 0.1, 0. , 0. , 0.3, 0. , 0.1, 0.2]])

In [30]:
# QNN backward pass, returns a tuple of sparse matrices
qnn4.backward(input4, weights4)

(None, array([[[-0.1 , -0.1 , -0.4 , -0.1 , -0.05, -0.3 ],
         [ 0.  ,  0.05,  0.  ,  0.1 ,  0.  , -0.05],
         [ 0.  ,  0.  ,  0.  ,  0.  , -0.1 ,  0.  ],
         [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.05],
         [-0.1 , -0.3 ,  0.3 , -0.15, -0.15,  0.35],
         [ 0.  ,  0.  ,  0.05,  0.1 , -0.1 ,  0.  ],
         [-0.2 ,  0.25,  0.  , -0.05,  0.2 ,  0.05],
         [ 0.4 ,  0.1 ,  0.05,  0.1 ,  0.2 ,  0.  ]]]))

In [31]:
# specify circuit QNN
parity = lambda x: "{:b}".format(x).count("1") % 2
output_shape = 2 # this is required in case of a callable with dense output
qnn6 = CircuitQNN(
    qc,
    [],
    qc.parameters,
    sparse=False,
    interpret=parity,
    output_shape=output_shape,
    quantum_instance=qi_qasm,
)

In [32]:
# define random input and weights
input6 = algorithm_globals.random.random(qnn6.num_inputs)
weights6 = algorithm_globals.random.random(qnn6.num_weights)

In [33]:
# QNN forward pass
qnn6.forward(input6, weights6)

array([[0.7, 0.3]])

In [34]:
# QNN backward pass
qnn6.backward(input6, weights6)

(None, array([[[-3.00000000e-01,  1.11022302e-16, -3.50000000e-01,
           1.50000000e-01,  1.00000000e-01, -3.00000000e-01],
         [ 3.00000000e-01,  0.00000000e+00,  3.50000000e-01,
          -1.50000000e-01, -1.00000000e-01,  3.00000000e-01]]]))

In [35]:
# specify circuit QNN
qnn7 = CircuitQNN(qc, [], qc.parameters, sampling=True, quantum_instance=qi_qasm)

In [36]:
# define random input and weights
input7 = algorithm_globals.random.random(qnn7.num_inputs)
weights7 = algorithm_globals.random.random(qnn7.num_weights)

In [37]:
# QNN forward pass, results in samples of measured bit strings mapped to integers
qnn7.forward(input7, weights7)

array([[[1.],
        [6.],
        [6.],
        [0.],
        [0.],
        [0.],
        [1.],
        [7.],
        [0.],
        [0.]]])

In [38]:
# QNN backward pass
qnn7.backward(input7, weights7)

(None, None)

In [39]:
# specify circuit QNN
qnn8 = CircuitQNN(qc, [], qc.parameters, sampling=True, interpret=parity, quantum_instance=qi_qasm)

In [40]:
# define random input and weights
input8 = algorithm_globals.random.random(qnn8.num_inputs)
weights8 = algorithm_globals.random.random(qnn8.num_weights)

In [41]:
# QNN forward pass, results in samples of measured bit strings
qnn8.forward(input8, weights8)

array([[[0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.]]])

In [42]:
# QNN backward pass
qnn8.backward(input8, weights8)

(None, None)